In [1]:
import torch
class DummyGPTModel(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = torch.nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = torch.nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = torch.nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = torch.nn.Sequential(
            *[DummyTransformerBlock(cfg)
            for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = torch.nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
    
    def forward(self, x):
        return x

class DummyLayerNorm(torch.nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    
    def forward(self, x):
        return x

In [2]:
import tiktoken
import torch

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [3]:
torch.manual_seed(123)
GPT_CONFIG_124M={
    "vocab_size": 50257,
    "context_length" : 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate":0.1,
    "qkv_bias": False
}
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("output_shape", logits.shape)

output_shape torch.Size([2, 4, 50257])


In [4]:
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
layer = torch.nn.Sequential(
    torch.nn.Linear(5, 2), torch.nn.ReLU()
)
out = layer(batch_example)
print(out)

mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)

out_norm = (out - mean) / torch.sqrt(var)
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print(out_norm)


tensor([[0.4490, 0.4633],
        [0.4363, 0.3558]], grad_fn=<ReluBackward0>)
tensor([[-0.7071,  0.7071],
        [ 0.7071, -0.7071]], grad_fn=<DivBackward0>)


In [5]:
class LayerNorm(torch.nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = torch.nn.Parameter(torch.ones(emb_dim))
        self.shift = torch.nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        
        x_norm = (x-mean) / torch.sqrt(var + self.eps)
        return self.scale * x_norm + self.shift

In [6]:
class GELU(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x *(1 + torch.tanh(
            torch.sqrt(torch.tensor(2/torch.pi)) * (x + 0.044715 * torch.pow(x, 3))
        ))

In [7]:
class FeedForward(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']),
            GELU(),
            torch.nn.Linear(4*cfg['emb_dim'], cfg['emb_dim'])
        )
    
    def forward(self, x):
        return self.layers(x)

In [8]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


In [23]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, bias, dropout, num_heads):
        super().__init__()
        assert (d_out % num_heads==0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.w_query = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=bias)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask", 
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.w_key(x)
        queries = self.w_query(x)
        values = self.w_value(x)
        
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        atten_score = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        atten_score.masked_fill_(mask_bool, -torch.inf)
        atten_weight = torch.softmax(atten_score / keys.shape[-1]**0.5, dim=-1)
        atten_weight = self.dropout(atten_weight)
        context_vec = (atten_weight @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        context_vec = self.out_proj(context_vec)
        return context_vec

In [24]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg['emb_dim'],
            d_out=cfg['emb_dim'],
            context_length=cfg['context_length'],
            num_heads=cfg['n_heads'],
            dropout=cfg['drop_rate'],
            bias=cfg['qkv_bias']
        )
        self.ffn = FeedForward(cfg)
        self.layernorm1 = LayerNorm(cfg['emb_dim'])
        self.layernorm2 = LayerNorm(cfg['emb_dim'])
        self.dropout=torch.nn.Dropout(cfg['drop_rate'])
        
    def forward(self, x):
        shortcut = x
        x = self.layernorm1(x)
        x = self.att(x)
        x = self.dropout(x)
        x = x + shortcut
        
        shortcut = x
        x = self.layernorm2(x)
        x = self.ffn(x)
        x = self.dropout(x)
        x = x+shortcut
        return x

In [25]:
class GPTModel(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = torch.nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = torch.nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = torch.nn.Dropout(cfg['drop_rate'])
        
        self.trf_blocks = torch.nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )
        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = torch.nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias=False
        )
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tokens_embedding = self.tok_emb(in_idx)
        pos_embedding = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tokens_embedding + pos_embedding
        x = self.drop_emb(x)
        
        x = self.trf_blocks(x)
        x  = self.final_norm(x)
        logits = self.out_head(x)
        return logits
        

In [26]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print(out)

tensor([[[-0.6632,  0.4968,  0.2568,  ..., -0.6531, -0.4572, -0.3779],
         [-0.3868, -0.5857, -0.7536,  ..., -0.4119, -0.2485, -0.1436],
         [ 0.3765,  0.1524, -0.7572,  ..., -0.2340, -1.0687, -0.1559],
         [-1.3663,  0.8720, -0.3075,  ...,  0.4330, -0.1335, -0.3234]],

        [[-0.6745,  0.1819,  0.2689,  ..., -0.4628, -0.6042, -0.4504],
         [-0.0142, -0.0486,  0.4236,  ...,  0.4378, -0.7191,  0.4629],
         [ 0.9519,  0.6377, -0.0618,  ..., -0.0209, -0.3523, -0.0424],
         [-0.2315,  0.2993,  0.6264,  ...,  0.7971, -0.7657, -0.2236]]],
       grad_fn=<UnsafeViewBackward0>)
